In [5]:
import numpy as np
from scipy import stats

Одно из условий применимости t-критерия Стьюдента — нормальность исходных данных (хотя на деле тест всё же устойчив к отклонениям).

Но давайте проверим, сломается ли что-то, если мы будем подавать в t-критерий Стьюдента данные не из нормального распределения.

Представим, что мы хотим сравнить скрипты, по которым обзваниваем наших клиентов с предложением о новой услуге. В результате клиенты либо соглашаются, либо нет. Хотим оценить, лучше ли новый скрипт, чем старый.

Для этого реализуйте функцию, которая будет генерировать две выборки одного размера из распределения Бернулли (пусть количество клиентов в группе равно 1000, а вероятность для генерации равна 0.2), оценивать с помощью t-критерия Стьюдента p-value для гипотезы о равенстве средних.

С помощью этой функции просимулируйте 1000-10000 раз A/A тест и посмотрите на распределение p-value. Также можно оценить долю ложноположительных ошибок. Можете ли вы сказать, что из-за неправильного применения t-критерия Стьюдента сломалась процедура A/B тестирования?

In [6]:
def generate_and_test(n=1000, p=0.2):
    # Генерация двух выборок из распределения Бернулли
    group1 = np.random.binomial(1, p, n)
    group2 = np.random.binomial(1, p, n)
    
    t_stat, p_value = stats.ttest_ind(group1, group2)
    
    return p_value

In [12]:
def simulate_test(num=10000):
    p_values = [generate_and_test() for i in range(num)]
    return np.array(p_values)

In [13]:
def count(values):
    return np.sum(values < 0.05)/len(values)

In [14]:
p_values = simulate_test()
print("Распределение p-value:")
print(p_values)

false_positive_rate = count(p_values)
print(f"Доля ложноположительных ошибок (alpha=0.05): {false_positive_rate:.4f}")

Распределение p-value:
[0.66071562 0.91247163 0.55191843 ... 0.62236045 0.739334   0.08735467]
Доля ложноположительных ошибок (alpha=0.05): 0.0499


Оцените минимальный размер выборки по формуле из лекции, если мы хотим иметь ошибку первого рода, равную 5%, мощность 90%, дисперсия нашей метрики равна 10, а эффект, который мы хотели бы детектировать, равен 0.5.

При использовании формулы вы заметите, что для вычисления вам уже нужно знать размер выборки (как число степеней свободы распределения Стьюдента), однако от этого значения не зависит порядок вычисленного числа, поэтому можете попробовать подставить разные значения (в том числе ближе к тому, которое вы в итоге будете получать). Величину какого порядка вы получили? Заметим, что это размер одной из групп при условии, что группы в нашем эксперименте одинакового размера.

In [18]:
import scipy.stats as stats
import math

def calculate_sample_size(alpha=0.05, power=0.90, sigma_squared=10, delta=0.5):
    z_alpha = stats.norm.ppf(1 - alpha / 2)
    z_beta = stats.norm.ppf(power)
    
    n = ((z_alpha + z_beta)**2 * 2 * sigma_squared) / delta**2
    
    return math.ceil(n) 


sample_size = calculate_sample_size()

print(f"Минимальный размер выборки: {sample_size}")


Минимальный размер выборки: 841


А теперь давайте попробуем оценить мощность с помощью симуляций, похожих на A/A эксперименты.

Предположим, что мы оцениваем, сколько времени пользователи проводят при изучении нашего рекламного предложения (сколько времени проходит от открытия до закрытия веб-страницы). До запуска эксперимента мы бы хотели понимать, какую мощность нам стоит ожидать, если мы будем запускать эксперимент на неделю.

Для этого аналогично процедуре проверки системы экспериментирования в A/A экспериментах реализуйте функцию, где будете генерировать две выборки из нормального распределения (предполагаем, что время распределено нормально со средним в контрольной группе 200 секунд и среднеквадратичным отклонением 30) одинакового размера (у нас 1000 пользователей в неделю, мы разбиваем их на 2 группы).

Повторите процедуру «эксперимента» 1000-10000 раз, оцените, в каком проценте случаев мы будем детектировать изменение в метрике, если в тестовой группе среднее будет равно 205 секунд. Это и будет мощностью в нашем эксперименте. В качестве статистического критерия используйте t-критерий Стьюдента, альтернатива двусторонняя, уровень значимости 0.05.

In [33]:
def generate(mean=200,mean_t = 205, std=30, n=500):
    group1 = np.random.normal(mean, std, n)
    group2 = np.random.normal(mean_t, std, n)
    
    t_stat, p_value = stats.ttest_ind(group1, group2)
    
    return p_value < 0.05

In [34]:
def simulate_power(num_simulations=10000):
    results = [generate() for _ in range(num_simulations)]
    power = np.mean(results) 
    return power

In [35]:
power = simulate_power()
print("оценка мощности:")
print(power)

оценка мощности:
0.7564


In [32]:
import numpy as np
from scipy import stats

# Параметры
mu_control = 200  # Среднее время в контрольной группе
mu_test = 205     # Среднее время в тестовой группе
sigma = 30        # Среднеквадратическое отклонение
n_users = 1000    # Количество пользователей в неделю
n_iterations = 10000  # Количество итераций для симуляции
alpha = 0.05      # Уровень значимости

# Функция для проведения одного эксперимента
def run_experiment():
    control_group = np.random.normal(mu_control, sigma, n_users // 2)
    test_group = np.random.normal(mu_test, sigma, n_users // 2)
    
    # t-критерий Стьюдента
    t_stat, p_value = stats.ttest_ind(control_group, test_group)
    
    # Проверяем, является ли p-value меньше уровня значимости
    return p_value < alpha

# Проводим эксперименты
results = [run_experiment() for _ in range(n_iterations)]

# Оцениваем мощность
power = np.mean(results)
print(f'Оцененная мощность эксперимента: {power:.4f}')


Оцененная мощность эксперимента: 0.7408
